#**Feature Engineering on Categorical Data**

**Import lib**

In [1]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import category_encoders as ce

In [ ]:
# uncomment when necessary
##upload the adult.data
# from google.colab import files
# uploaded = files.upload()

The purpose of this part is to show how to perform different ways of feature engineering on categorical data and the effectiveness of doing them.

4 cases below are all based on the same data(adult.data from https://archive.ics.uci.edu/dataset/2/adult ) to perform a classification of income class based on difference senarios.

**Case 1: Doing classfication directly (without feature engineering on categorical data and removing them)**

In [27]:
# predefine the column names
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
    'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income'
]

# import dataset
df = pd.read_csv('/content/adult.data', delimiter=',',header=None, names = column_names)

#the purpose of dropping this column is to apply ordinal encoding in the later stage based on the 'education' column, to make sure consistency, do it for all situations
df = df.drop('education_num', axis=1)

# remove the unknow country
df = df[df['native_country'] != ' ?']

#showdf, presenting purpose
df


age          workclass  fnlwgt    education       marital_status  \
0       39          State-gov   77516    Bachelors        Never-married   
1       50   Self-emp-not-inc   83311    Bachelors   Married-civ-spouse   
2       38            Private  215646      HS-grad             Divorced   
3       53            Private  234721         11th   Married-civ-spouse   
4       28            Private  338409    Bachelors   Married-civ-spouse   
...    ...                ...     ...          ...                  ...   
32556   27            Private  257302   Assoc-acdm   Married-civ-spouse   
32557   40            Private  154374      HS-grad   Married-civ-spouse   
32558   58            Private  151910      HS-grad              Widowed   
32559   22            Private  201490      HS-grad        Never-married   
32560   52       Self-emp-inc  287927      HS-grad   Married-civ-spouse   

               occupation    relationship    race      sex  capital_gain  \
0            Adm-clerical   Not-in-family   White     Male          2174   
1         Exec-managerial         Husband   White     Male             0   
2       Handlers-cleaners   Not-in-family   White     Male             0   
3       Handlers-cleaners         Husband   Black     Male             0   
4          Prof-specialty            Wife   Black   Female             0   
...                   ...             ...     ...      ...           ...   
32556        Tech-support            Wife   White   Female             0   
32557   Machine-op-inspct         Husband   White     Male             0   
32558        Adm-clerical       Unmarried   White   Female             0   
32559        Adm-clerical       Own-child   White     Male             0   
32560     Exec-managerial            Wife   White   Female         15024   

       capital_loss  hours_per_week  native_country  income  
0                 0              40   United-States   <=50K  
1                 0              13   United-States   <=50K  
2                 0              40   United-States   <=50K  
3                 0              40   United-States   <=50K  
4                 0              40            Cuba   <=50K  
...             ...             ...             ...     ...  
32556             0              38   United-States   <=50K  
32557             0              40   United-States    >50K  
32558             0              40   United-States   <=50K  
32559             0              20   United-States   <=50K  
32560             0              40   United-States    >50K  

[31978 rows x 14 columns]

###**Note: here will result in error for illustrating purpose of what happen if we do not do feature engineering on categorical data, need to mannul run the rest of code (uncomment the whole cell to run )**

In [28]:
# #here will result in error for illustrating purpose of what happen if we do not do feature engineering on categorical data
# X = df.drop('income', axis=1)
# y = df['income']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a random forest classifier
# clf = RandomForestClassifier(random_state=42)
# clf.fit(X_train, y_train)

# # Evaluate the model accuracy without feature engineering
# y_pred = clf.predict(X_test)
# accuracy_without_feature_engineering = accuracy_score(y_test, y_pred)

# print("Accuracy without feature engineering:", accuracy_without_feature_engineering)

**Case2: doing classification without categorical data**
> not practical and not recommend

>it somehow works in this case, but think of the situations where you have 10 columns in total and 7 of them are categorical, you will  **only have 2 columns left** to predict the target


In [29]:
#reload data before every new case

# predefine the column names
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
    'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income'
]

# import dataset
df = pd.read_csv('/content/adult.data', delimiter=',',header=None, names = column_names)

#the purpose of dropping this column is to apply ordinal encoding in the later stage based on the 'education' column, to make sure consistency, do it for all situations
df = df.drop('education_num', axis=1)

# remove the unknow country
df = df[df['native_country'] != ' ?']


In [30]:
#to do classfication only with those given numerical data
df_numerical = df[['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week']]

#split the data into feastures (X) and target (y)
X = df_numerical
y = df['income']

#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train a random forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

#evaluate the model accuracy using only numerical columns
y_pred = clf.predict(X_test)
accuracy_without_categorical_data = accuracy_score(y_test, y_pred)

print("Accuracy without categorical data :", accuracy_without_categorical_data)

Accuracy without categorical data : 0.7698561601000625


**Case3: doing classfication with feature engineering on categorical data (One-Hot Encoding for All Categorical Data)**

In [31]:
#reload data before every new case

# predefine the column names
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
    'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income'
]

# import dataset
df = pd.read_csv('/content/adult.data', delimiter=',',header=None, names = column_names)

#the purpose of dropping this column is to apply ordinal encoding in the later stage based on the 'education' column, to make sure consistency, do it for all situations
df = df.drop('education_num', axis=1)

# remove the unknow country
df = df[df['native_country'] != ' ?']


In [32]:
X = df.drop('income', axis=1)
y = df['income']

#one-hot encoding for categorical data
X = pd.get_dummies(X, columns=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country'])

#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train a random forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

#evaluate the model accuracy
y_pred = clf.predict(X_test)
accuracy_with_feature_engineering = accuracy_score(y_test, y_pred)

print("Accuracy with feature engineering:", accuracy_with_feature_engineering)

Accuracy with feature engineering: 0.8563164477798624


In [33]:

#put the original sex column and sex_encoded column togather to better see the comparison
sex_compare = pd.merge(df[['sex']], X[['sex_ Male', 'sex_ Female']], left_index=True, right_index=True)

#show the first 10 observations
sex_compare.head(10)

sex  sex_ Male  sex_ Female
0     Male          1            0
1     Male          1            0
2     Male          1            0
3     Male          1            0
4   Female          0            1
5   Female          0            1
6   Female          0            1
7     Male          1            0
8   Female          0            1
9     Male          1            0

In [34]:
#notice that since we applied the one-hot encoding to all the categorical data, there is a significant increase in the column numbers from 14 to 107
print(f'The original datafame has {df.shape[1]} columns')
print(f'The encoded datafame has {X.shape[1]} columns')

The original datafame has 14 columns
The encoded datafame has 106 columns


**Case4: doing classfication with feature engineering on categorical data (Treat them using different methods)**

In [49]:
#reload data before every new case

# predefine the column names
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
    'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income'
]

# import dataset
df = pd.read_csv('/content/adult.data', delimiter=',',header=None, names = column_names)

#the purpose of dropping this column is to apply ordinal encoding in the later stage based on the 'education' column, to make sure consistency, do it for all situations
df = df.drop('education_num', axis=1)

# remove the unknow country
df = df[df['native_country'] != ' ?']


In [50]:
# Check the complexity of each categoical data (asked ChatGPT)
for column in df.columns:
    if df[column].dtype == 'object':
        unique_count = df[column].nunique()
        print(f"Column '{column}' has {unique_count} unique values.")

#The results show that column like native_country has various unique values, if we perform a One-Hot encoding
#It will significantly increase the dimensionality of the dataset

Column 'workclass' has 9 unique values.
Column 'education' has 16 unique values.
Column 'marital_status' has 7 unique values.
Column 'occupation' has 15 unique values.
Column 'relationship' has 6 unique values.
Column 'race' has 5 unique values.
Column 'sex' has 2 unique values.
Column 'native_country' has 41 unique values.
Column 'income' has 2 unique values.


In [51]:
#In order to perform target encoding (Example 6), we need the target column(income) in this particular example to be numerical as well
# Therefore, we create a new column called new_income, if the income is  ' >50K', mark it as 1, else 0
df['new_income'] = df.income.map(lambda x : 1 if  x == ' >50K' else 0)

In [54]:

#define categorical columns for different types of featurn engineering methods
label_encoded_column = ['marital_status']
one_hot_encoded_columns = ['sex']
ordinal_encoded_columns = ['education']
frequency_encoded_columns = ['workclass', 'race']
binary_encoded_columns = ['occupation', 'relationship']

#define target encoded columns
target_encoded_columns = ['native_country']

#copy the dataframe to preserve the original data
df_encoded = df.copy()

# Example1: Label encoding (will replace the original column with numercial numbers)
label_encoder = LabelEncoder()
df_encoded['marital_status'] = label_encoder.fit_transform(df_encoded['marital_status'])

# Example 2: One_hot encoding(will drop the original column)
df_encoded = pd.get_dummies(df_encoded, columns=one_hot_encoded_columns)

# Example 3: Ordinal encoding
education_order = [
    ' Preschool', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' 10th', ' 11th',
    ' 12th', ' HS-grad', ' Some-college', ' Assoc-acdm', ' Assoc-voc',
    ' Bachelors', ' Masters', ' Doctorate', ' Prof-school'
]

df_encoded['education_encoded'] = df_encoded['education'].apply(lambda x: education_order.index(x))
# Drop the original 'education' column
df_encoded = df_encoded.drop(columns=['education'])

# df_encoded = df_encoded.drop(columns=['workclass', 'native_country'])

# Example 4: Frequency encoding
for column in frequency_encoded_columns:
    frequency_column_name = f'{column}_frequency'
    df_encoded[frequency_column_name] = df_encoded[column].map(df_encoded[column].value_counts())

# Example 5: Binary encoding
binary_encoder = ce.BinaryEncoder(cols=binary_encoded_columns)
df_encoded = binary_encoder.fit_transform(df_encoded)

#Example 6: Target encoding
target_encoder = ce.TargetEncoder(cols=target_encoded_columns)
df_encoded = target_encoder.fit_transform(df_encoded, df_encoded['new_income'])

#drop the columns in frequency_encoded_columns
df_encoded = df_encoded.drop(columns=frequency_encoded_columns)


# df_encoded
# Split the data into features (X) and target (y)
#drop both income and newincome column to avoid inforamation exposure
X = df_encoded.drop(['income','new_income'], axis=1)
y = df_encoded['income']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model accuracy
y_pred = clf.predict(X_test)
accuracy_with_feature_engineering = accuracy_score(y_test, y_pred)

print("Accuracy with feature engineering with different methods :", accuracy_with_feature_engineering)

Accuracy with feature engineering with different methods : 0.8599124452782989


**Comparison before and after each method for featur engineering**

In [39]:
#Example 1: label encoding , columns comparison before and after feature engineering

marital_status_compare =  pd.merge(df[['marital_status']],X[['marital_status']], left_index=True, right_index=True)
marital_status_compare.head(10)

marital_status_x  marital_status_y
0           Never-married                 4
1      Married-civ-spouse                 2
2                Divorced                 0
3      Married-civ-spouse                 2
4      Married-civ-spouse                 2
5      Married-civ-spouse                 2
6   Married-spouse-absent                 3
7      Married-civ-spouse                 2
8           Never-married                 4
9      Married-civ-spouse                 2

In [40]:
#Example 2 : one hot encoding , columns comparison before and after feature engineering
sex_compare = pd.merge(df[['sex']], X[['sex_ Male', 'sex_ Female']], left_index=True, right_index=True)
sex_compare.head(10)

sex  sex_ Male  sex_ Female
0     Male          1            0
1     Male          1            0
2     Male          1            0
3     Male          1            0
4   Female          0            1
5   Female          0            1
6   Female          0            1
7     Male          1            0
8   Female          0            1
9     Male          1            0

In [41]:
#Example 3: Ordinal encoding, columns comparison before and after feature engineering
edu_compare = pd.merge(df[['education']],X[['education_encoded']], left_index=True, right_index=True)
edu_compare.head(10)

education  education_encoded
0   Bachelors                 12
1   Bachelors                 12
2     HS-grad                  8
3        11th                  6
4   Bachelors                 12
5     Masters                 13
6         9th                  4
7     HS-grad                  8
8     Masters                 13
9   Bachelors                 12

In [42]:
# Example 4.1 work class: Frequency encoding, columns comparison before and after feature engineering
workclass_compare = pd.merge(df[['workclass']],X[['workclass_frequency']], left_index=True, right_index=True)
workclass_compare.head(10)

workclass  workclass_frequency
0          State-gov                 1279
1   Self-emp-not-inc                 2499
2            Private                22286
3            Private                22286
4            Private                22286
5            Private                22286
6            Private                22286
7   Self-emp-not-inc                 2499
8            Private                22286
9            Private                22286

In [43]:
# Example 4.2 race: Frequency encoding, columns comparison before and after feature engineering
race_compare = pd.merge(df[['race']],X[['race_frequency']], left_index=True, right_index=True)
race_compare.head(10)

race  race_frequency
0   White           27430
1   White           27430
2   White           27430
3   Black            3028
4   Black            3028
5   White           27430
6   Black            3028
7   White           27430
8   White           27430
9   White           27430

In [44]:
# Example 5.1 occupation : Binary encoding columns comparison before and after feature engineering
occupation_compare = pd.merge(df[['occupation']],X[['occupation_0', 'occupation_1', 'occupation_2','occupation_3']], left_index=True, right_index=True)
occupation_compare.head(10)


occupation  occupation_0  occupation_1  occupation_2  occupation_3
0        Adm-clerical             0             0             0             1
1     Exec-managerial             0             0             1             0
2   Handlers-cleaners             0             0             1             1
3   Handlers-cleaners             0             0             1             1
4      Prof-specialty             0             1             0             0
5     Exec-managerial             0             0             1             0
6       Other-service             0             1             0             1
7     Exec-managerial             0             0             1             0
8      Prof-specialty             0             1             0             0
9     Exec-managerial             0             0             1             0

In [45]:
# Example 5.2 relationship : Binary encoding, columns comparison before and after feature engineering
relationship_compare = pd.merge(df[['relationship']],X[['relationship_0', 'relationship_1', 'relationship_2']], left_index=True, right_index=True)
relationship_compare.head(10)


relationship  relationship_0  relationship_1  relationship_2
0   Not-in-family               0               0               1
1         Husband               0               1               0
2   Not-in-family               0               0               1
3         Husband               0               1               0
4            Wife               0               1               1
5            Wife               0               1               1
6   Not-in-family               0               0               1
7         Husband               0               1               0
8   Not-in-family               0               0               1
9         Husband               0               1               0

In [46]:
#Example 6 native_country: target encoding,columns comparison before and after feature engineering
country_compare = pd.merge(df[['native_country']],X[['native_country']], left_index=True, right_index=True)
country_compare.head(10)

native_country_x  native_country_y
0    United-States          0.245835
1    United-States          0.245835
2    United-States          0.245835
3    United-States          0.245835
4             Cuba          0.263145
5    United-States          0.245835
6          Jamaica          0.123719
7    United-States          0.245835
8    United-States          0.245835
9    United-States          0.245835

In [48]:
# How do we get the number for the values in the above native_country_y?
#here is a little bit breakdown/insights of what the target_encoder does
target_encoding_mapping = df.groupby('native_country')['new_income'].mean()
target_encoding_mapping

native_country
 Cambodia                      0.368421
 Canada                        0.322314
 China                         0.266667
 Columbia                      0.033898
 Cuba                          0.263158
 Dominican-Republic            0.028571
 Ecuador                       0.142857
 El-Salvador                   0.084906
 England                       0.333333
 France                        0.413793
 Germany                       0.321168
 Greece                        0.275862
 Guatemala                     0.046875
 Haiti                         0.090909
 Holand-Netherlands            0.000000
 Honduras                      0.076923
 Hong                          0.300000
 Hungary                       0.230769
 India                         0.400000
 Iran                          0.418605
 Ireland                       0.208333
 Italy                         0.342466
 Jamaica                       0.123457
 Japan                         0.387097
 Laos                    

References:

1. https://www.datacamp.com/tutorial/categorical-data
2. https://dotdata.com/blog/feature-engineering-for-categorical-attributes/
3.https://medium.com/geekculture/feature-engineering-for-categorical-data-a77a04b3308

Dataset is from:
https://archive.ics.uci.edu/dataset/2/adult
